In [208]:
# imports

import pandas as pd
import requests
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [209]:
# Load csv file from local machine

data = pd.read_csv('C:\\Users\\affuy\\Documents\\Data_Sets\\df_2.csv')

# View make up and structure of the file
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   394 non-null    int64  
 1   empty_slots  394 non-null    int64  
 2   extra        394 non-null    object 
 3   free_bikes   394 non-null    int64  
 4   id           394 non-null    object 
 5   latitude     394 non-null    float64
 6   longitude    394 non-null    float64
 7   name         394 non-null    object 
 8   timestamp    394 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 27.8+ KB


In [210]:
# Visualize the DataFrame
data

,Unnamed: 0,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp
0,0,17,"{'address': 'Calle Miguel Moya nº 1,', 'light'...",10,82e78d26829a2dc89122e8f0988e2a28,40.420589,-3.705842,2 - Miguel Moya,2023-09-05T01:44:12.473000Z
1,1,9,{'address': 'Plaza del Conde del Valle de Súch...,10,248a049595da65f6aada65476ca1f958,40.430294,-3.706917,3 - Plaza Conde Suchil,2023-09-05T01:44:12.638000Z
2,2,17,"{'address': 'Calle Hortaleza nº 63,', 'light':...",2,bb6afabc8fbc0e12d4717d6a3a3473c3,40.424148,-3.698447,6 - Colegio Arquitectos,2023-09-05T01:44:12.626000Z
3,3,19,"{'address': 'Calle Hortaleza nº 75,', 'light':...",0,55a618a12ce76d28b82343f2c60d6693,40.425191,-3.697771,7 - Hortaleza,2023-09-05T01:44:12.474000Z
4,4,17,"{'address': 'Plaza de San Miguel nº 9,', 'ligh...",7,5ff854ab96bd68ae3c28be05a0dab949,40.415606,-3.709508,9 - Plaza de San Miguel,2023-09-05T01:44:12.630000Z
...,...,...,...,...,...,...,...,...,...
389,389,12,"{'address': 'Arturo Soria, 329,', 'light': 'ye...",11,dc7b27b928b39aea661e2a78c0dbc809,40.480386,-3.667248,"494 - Arturo Soria, 330",2023-09-05T01:44:12.612000Z
390,390,11,"{'address': 'Calle del tejo,', 'light': 'yello...",12,d394021b285e7bf3183a483e530d9daa,40.398001,-3.678027,272 - Calle del tejo,2023-09-05T01:44:12.949000Z
391,391,15,"{'address': ' Calle Babilonia, 19,', 'light': ...",8,2e175e4613adf44134fda53c82f326f5,40.467530,-3.588235,"559 - Calle Babilonia, 19",2023-09-05T01:44:12.957000Z
392,392,16,"{'address': 'Calle Albelda, 6,', 'light': 'yel...",8,7b1d1679aa52ec836f5d709292a8d177,40.449422,-3.657367,"528 - Albelda, 6",2023-09-05T01:44:12.958000Z


In [211]:
# Create a new column concatenating latitude and longitude into a comma-separated string
data['ll'] = data['latitude'].astype(str) + ',' + data['longitude'].astype(str)
data.head()


,Unnamed: 0,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp,ll
0,0,17,"{'address': 'Calle Miguel Moya nº 1,', 'light'...",10,82e78d26829a2dc89122e8f0988e2a28,40.420589,-3.705842,2 - Miguel Moya,2023-09-05T01:44:12.473000Z,"40.4205886,-3.7058415"
1,1,9,{'address': 'Plaza del Conde del Valle de Súch...,10,248a049595da65f6aada65476ca1f958,40.430294,-3.706917,3 - Plaza Conde Suchil,2023-09-05T01:44:12.638000Z,"40.4302937,-3.7069171"
2,2,17,"{'address': 'Calle Hortaleza nº 63,', 'light':...",2,bb6afabc8fbc0e12d4717d6a3a3473c3,40.424148,-3.698447,6 - Colegio Arquitectos,2023-09-05T01:44:12.626000Z,"40.424148,-3.698447"
3,3,19,"{'address': 'Calle Hortaleza nº 75,', 'light':...",0,55a618a12ce76d28b82343f2c60d6693,40.425191,-3.697771,7 - Hortaleza,2023-09-05T01:44:12.474000Z,"40.4251906,-3.6977715"
4,4,17,"{'address': 'Plaza de San Miguel nº 9,', 'ligh...",7,5ff854ab96bd68ae3c28be05a0dab949,40.415606,-3.709508,9 - Plaza de San Miguel,2023-09-05T01:44:12.630000Z,"40.4156057,-3.7095084"


In [212]:
# create a new dataframe with required columns
data2 = data[['name', 'latitude', 'longitude', 'll']]
data2

,name,latitude,longitude,ll
0,2 - Miguel Moya,40.420589,-3.705842,"40.4205886,-3.7058415"
1,3 - Plaza Conde Suchil,40.430294,-3.706917,"40.4302937,-3.7069171"
2,6 - Colegio Arquitectos,40.424148,-3.698447,"40.424148,-3.698447"
3,7 - Hortaleza,40.425191,-3.697771,"40.4251906,-3.6977715"
4,9 - Plaza de San Miguel,40.415606,-3.709508,"40.4156057,-3.7095084"
...,...,...,...,...
389,"494 - Arturo Soria, 330",40.480386,-3.667248,"40.48038551,-3.66724784"
390,272 - Calle del tejo,40.398001,-3.678027,"40.3980005,-3.6780275"
391,"559 - Calle Babilonia, 19",40.467530,-3.588235,"40.4675299,-3.5882347"
392,"528 - Albelda, 6",40.449422,-3.657367,"40.4494225,-3.6573666"


In [213]:
import requests
import os
import pandas as pd

# Send request with proveided requirements

api_key = os.environ["FOURSQUARE_API_KEY"]

# Initialize an empty list to store results
results = []

# Iterate through the rows of the 'data2' DataFrame
for index, row in data2.iterrows():
    ll = row['ll']
    radius = 1000  # You can adjust the radius as needed
    categories = 13000  # You can adjust the category as needed
    limit = 50  # You can adjust the limit as needed

    url = f"https://api.foursquare.com/v3/places/search?ll={ll}&radius={radius}&categories={categories}&sort=RATING&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": api_key
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data2_result = response.json()
        results.append(data2_result)
    else:
        print(f"Failed to retrieve data for ll: {ll}. Status code: {response.status_code}")

# Combine the results into a single DataFrame if needed
# Assuming each result is a DataFrame, you can use pandas' concat method
# combined_results = pd.concat([pd.DataFrame(result) for result in results], ignore_index=True)


In [214]:
# View output of request
data2_result

{'results': [{'fsq_id': '5e149ca13a952d000805626b',
   'categories': [{'id': 13049,
     'name': 'Diner',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 388,
   'geocodes': {'main': {'latitude': 40.446518, 'longitude': -3.727189}},
   'link': '/v3/places/5e149ca13a952d000805626b',
   'location': {'address': 'Calle de Carranza, 19',
    'admin_region': 'Comunidad de Madrid',
    'country': 'ES',
    'cross_street': '',
    'formatted_address': 'Calle de Carranza, 19, 28004 Madrid Madrid',
    'locality': 'Madrid',
    'postcode': '28004',
    'region': 'Madrid'},
   'name': 'Igo Waffle Fuencarral',
   'related_places': {},
   'timezone': 'Europe/Madrid'},
  {'fsq_id': '4bf13e8f6f8aa593fca2c23c',
   'categories': [{'id': 13065,
     'name': 'Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 649,
   

In [215]:
# View type
print(type(data2_result))

<class 'dict'>


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [216]:
import pandas as pd

def extract_foursquare_data(data2_result):
    # Initialize empty lists to store extracted data
    fsq_ids = []
    categories = []
    distance = []
    latitude = []
    longitude = []
    address = []
    country = []
    locality = []

    for result in data2_result['results']:
        fsq_id = result.get('fsq_id', None)
        
        # Extract category names as a comma-separated string
        category_names = ', '.join([cat['name'] for cat in result['categories']])
        
        dist = result.get('distance', None)
        lat = result['geocodes']['main']['latitude']
        lon = result['geocodes']['main']['longitude']
        addr = result.get('location', {}).get('address', None)
        cntry = result.get('location', {}).get('country', None)
        loc = result.get('location', {}).get('locality', None)
        
        # Append extracted data to respective lists
        fsq_ids.append(fsq_id)
        categories.append(category_names)
        distance.append(dist)
        latitude.append(lat)
        longitude.append(lon)
        address.append(addr)
        country.append(cntry)
        locality.append(loc)

    # Create a DataFrame from the extracted data
    data_frame = pd.DataFrame({
        'Foursquare ID': fsq_ids,
        'Categories': categories,
        'Distance': distance,
        'Latitude': latitude,
        'Longitude': longitude,
        'Address': address,
        'Country': country,
        'Locality': locality
    })
    
    return data_frame

# Example usage:
extracted_data_fs = extract_foursquare_data(data2_result)
print(extracted_data_fs)


              Foursquare ID              Categories  Distance   Latitude  \
0  5e149ca13a952d000805626b                   Diner       388  40.446518   
1  4bf13e8f6f8aa593fca2c23c              Restaurant       649  40.444758   
2  ddaeb85f643b4c81154ff48a  Restaurant, Restaurant       672  40.443393   
3  529ddb9e11d2a02e5dfccaec               Cafeteria       682  40.451593   
4  50bc9656e4b0198964a88bca             Coffee Shop       925  40.446184   
5  57d11afd498e864532c6a11e                Pizzeria       942  40.444596   

   Longitude                              Address Country Locality  
0  -3.727189                Calle de Carranza, 19      ES   Madrid  
1  -3.725214             Plaza Ramón y Cajal, s/n      ES   Madrid  
2  -3.727321               Avenida Complutense, 1      ES   Madrid  
3  -3.737784             Calle Juan del Rosal, 14      ES   Madrid  
4  -3.740853                   Pintor el Greco, 4      ES   Madrid  
5  -3.720932  Paseo de San Francisco de Sales, 23    

Put your parsed results into a DataFrame

In [217]:
# Create a DataFrame from the extracted data
extracted_data_fs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Foursquare ID  6 non-null      object 
 1   Categories     6 non-null      object 
 2   Distance       6 non-null      int64  
 3   Latitude       6 non-null      float64
 4   Longitude      6 non-null      float64
 5   Address        6 non-null      object 
 6   Country        6 non-null      object 
 7   Locality       6 non-null      object 
dtypes: float64(2), int64(1), object(5)
memory usage: 512.0+ bytes


In [218]:
# Save this as a csv file

import pandas as pd

# Save the DataFrame as a CSV file
extracted_data_fs.to_csv('C:\\Users\\affuy\\Documents\\Data_Sets\\df_foursquare.csv', index=False)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [219]:
import requests
import os
import pandas as pd

# Assuming you have the 'data2' DataFrame with 'latitude' and 'longitude' columns

yelp_api_key = os.environ["YELP_API_KEY"]

# Initialize an empty list to store results
yelp_results = []

# Iterate through the rows of the 'data2' DataFrame
for index, row in data2.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    radius = 1000  # You can adjust the radius as needed
    categories = "restaurants"  # You can adjust the category as needed
    limit = 50  # You can adjust the limit as needed

    url = f"https://api.yelp.com/v3/businesses/search"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "categories": categories,
        "limit": limit
    }

    headers = {
        "Authorization": f"Bearer {yelp_api_key}"
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        yelp_data = response.json()
        yelp_results.append(yelp_data)
    else:
        print(f"Failed to retrieve data for latitude: {latitude}, longitude: {longitude}. Status code: {response.status_code}")

# Process and extract relevant information from 'yelp_results' as needed
# You can follow a similar approach as in the previous Foursquare example


In [220]:
yelp_data

{'businesses': [{'id': 'Vc4RIVJO1H3fEVSxKhEyIQ',
   'alias': 'cafetería-facultad-farmacia-madrid-2',
   'name': 'Cafetería Facultad Farmacia',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/1WMLi0vxdTX8QSdWLY_pBQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/cafeter%C3%ADa-facultad-farmacia-madrid-2?adjust_creative=HtGBEqBv3zF4YDHhPbDfHg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=HtGBEqBv3zF4YDHhPbDfHg',
   'review_count': 2,
   'categories': [{'alias': 'cafeteria', 'title': 'Cafeteria'}],
   'rating': 3.5,
   'coordinates': {'latitude': 40.445423556744,
    'longitude': -3.72487552464008},
   'transactions': [],
   'price': '€',
   'location': {'address1': 'Plaza Ramón y Cajal, s/n',
    'address2': '',
    'address3': '',
    'city': 'Madrid',
    'zip_code': '28040',
    'country': 'ES',
    'state': 'M',
    'display_address': ['Plaza Ramón y Cajal, s/n', '28040 Madrid', 'Spain']},
   'phone': '+34913941734',
   'display_pho

In [221]:
print(type(yelp_data))

<class 'dict'>


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [222]:
import pandas as pd

def extract_yelp_data(yelp_data):
    # Initialize empty lists to store extracted data
    yelp_ids = []
    categories = []
    distance = []
    latitude = []
    longitude = []
    address = []
    city = []
    name = []
    rating = []

    businesses = yelp_data.get('businesses', [])

    if not businesses:
        print("No businesses found in the input data.")
        return None

    for business in businesses:
        yelp_id = business.get('id', None)
        category_names = ', '.join([cat['title'] for cat in business.get('categories', [])])
        dist = business.get('distance', None)
        lat = business['coordinates']['latitude']
        lon = business['coordinates']['longitude']
        addr = ', '.join(business.get('location', {}).get('display_address', []))
        city_name = business.get('location', {}).get('city', None)
        business_name = business.get('name', None)
        business_rating = business.get('rating', None)

        # Append extracted data to respective lists
        yelp_ids.append(yelp_id)
        categories.append(category_names)
        distance.append(dist)
        latitude.append(lat)
        longitude.append(lon)
        address.append(addr)
        city.append(city_name)
        name.append(business_name)
        rating.append(business_rating)

    # Create a DataFrame from the extracted data
    data_frame = pd.DataFrame({
        'Yelp ID': yelp_ids,
        'Categories': categories,
        'Distance': distance,
        'Latitude': latitude,
        'Longitude': longitude,
        'Address': address,
        'City': city,
        'Name': name,
        'Rating': rating
    })

    return data_frame

# Example usage:
extracted_data_yelp = extract_yelp_data(yelp_data)
if extracted_data_yelp is not None:
    print(extracted_data_yelp)


                  Yelp ID                        Categories     Distance  \
0  Vc4RIVJO1H3fEVSxKhEyIQ                         Cafeteria   616.451917   
1  BRA1z3dUr3mydVH3GGGmdQ                      Coffee & Tea   799.139855   
2  eQb3XqyhyuhFjeJWVZXIVw                         Cafeteria   388.335995   
3  ihZ_5G2_q0-QpDHZO0bJBw     Cafeteria, Breakfast & Brunch   692.005373   
4  F-yMUss08kAYNMFQ5pzvFw  Coffee & Tea, Breakfast & Brunch   675.972340   
5  2kF_1PF3OUAwDKYQjEw64A                      Coffee & Tea  1054.363331   
6  81FxHLD2sbLP_HPaG2XiJA                    Latin American  1051.327423   
7  0CchsiR66o0mpb5EdrNdpA                        Tapas Bars  1051.327423   

    Latitude  Longitude                                            Address  \
0  40.445424  -3.724876      Plaza Ramón y Cajal, s/n, 28040 Madrid, Spain   
1  40.442676  -3.725866           Avenida Complutense, 28040 Madrid, Spain   
2  40.445956  -3.728114      Avenida Complutense, S/N, 28040 Madrid, Spain   
3  

In [223]:
import pandas as pd

def extract_yelp_data(yelp_data):
    # Initialize empty lists to store extracted data
    yelp_ids = []
    categories = []
    distance = []
    latitude = []
    longitude = []
    address = []
    city = []
    name = []
    rating = []

    for business in yelp_data.get('businesses', []):
        yelp_id = business['id']
        category_names = ', '.join([cat['title'] for cat in business.get('categories', [])])
        dist = business.get('distance', None)
        lat = business['coordinates']['latitude']
        lon = business['coordinates']['longitude']
        addr = ', '.join(business.get('location', {}).get('display_address', []))
        city_name = business.get('location', {}).get('city', None)
        business_name = business.get('name', None)
        business_rating = business.get('rating', None)

        # Append extracted data to respective lists
        yelp_ids.append(yelp_id)
        categories.append(category_names)
        distance.append(dist)
        latitude.append(lat)
        longitude.append(lon)
        address.append(addr)
        city.append(city_name)
        name.append(business_name)
        rating.append(business_rating)

    # Create a DataFrame from the extracted data
    data_frame = pd.DataFrame({
        'Yelp ID': yelp_ids,
        'Categories': categories,
        'Distance': distance,
        'Latitude': latitude,
        'Longitude': longitude,
        'Address': address,
        'City': city,
        'Name': name,
        'Rating': rating
    })

    return data_frame

# Example usage:
extracted_data_yelp = extract_yelp_data(yelp_data)
print(extracted_data_yelp)


                  Yelp ID                        Categories     Distance  \
0  Vc4RIVJO1H3fEVSxKhEyIQ                         Cafeteria   616.451917   
1  BRA1z3dUr3mydVH3GGGmdQ                      Coffee & Tea   799.139855   
2  eQb3XqyhyuhFjeJWVZXIVw                         Cafeteria   388.335995   
3  ihZ_5G2_q0-QpDHZO0bJBw     Cafeteria, Breakfast & Brunch   692.005373   
4  F-yMUss08kAYNMFQ5pzvFw  Coffee & Tea, Breakfast & Brunch   675.972340   
5  2kF_1PF3OUAwDKYQjEw64A                      Coffee & Tea  1054.363331   
6  81FxHLD2sbLP_HPaG2XiJA                    Latin American  1051.327423   
7  0CchsiR66o0mpb5EdrNdpA                        Tapas Bars  1051.327423   

    Latitude  Longitude                                            Address  \
0  40.445424  -3.724876      Plaza Ramón y Cajal, s/n, 28040 Madrid, Spain   
1  40.442676  -3.725866           Avenida Complutense, 28040 Madrid, Spain   
2  40.445956  -3.728114      Avenida Complutense, S/N, 28040 Madrid, Spain   
3  

In [224]:
extracted_data_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Yelp ID     8 non-null      object 
 1   Categories  8 non-null      object 
 2   Distance    8 non-null      float64
 3   Latitude    8 non-null      float64
 4   Longitude   8 non-null      float64
 5   Address     8 non-null      object 
 6   City        8 non-null      object 
 7   Name        8 non-null      object 
 8   Rating      8 non-null      float64
dtypes: float64(4), object(5)
memory usage: 704.0+ bytes


Put your parsed results into a DataFrame

In [225]:
# Already parsed into a dataframe in the function. # Create a DataFrame from the extracted data
# data_frame = pd.DataFrame({ 'Yelp ID': yelp_ids,'Categories': categories,'Distance': distance,'Latitude': latitude,'Longitude': longitude,
# 'Address': address,'City': city,'Name': name,'Rating': rating})'''

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

##### I have found that both the Foursquare and Yelp APIs provide valuable data for various purposes. The choice of which one to use depends on your specific needs and the type of data you require. 

##### The Yelp API has been particularly helpful for me. It provided me with information primarily related to businesses and places, with a strong focus on user-generated reviews and ratings. This includes detailed information about businesses such as their names, addresses, contact details, ratings, reviews, and categories. It has proven to be very useful when I needed to analyze and compare user opinions, ratings, and reviews about restaurants, bars, and other businesses. I've found it to be more suitable for applications related to the hospitality and food service industry.

##### On the other hand, the Foursquare API has also been quite valuable in my work. It offers a broader range of location-based data and insights beyond just businesses and places. In addition to business details, Foursquare has provided me with data related to check-ins, user-generated tips, user profiles, and location trends. This extensive dataset has been instrumental in exploring user behavior, tracking trends in check-in activity, and understanding user preferences in a more general sense. It's a versatile choice for applications that require a deeper understanding of user engagement and location-based behavior.

##### In summary, if you primarily need detailed information about businesses, their ratings, and reviews, the Yelp API may be more suitable for your needs. On the other hand, if you're interested in a broader range of location-based data, including user activity and trends, the Foursquare API can provide valuable insights. Your choice should align with your specific analytical goals and the type of data that best serves your analysis."

Get the top 10 restaurants according to their rating

In [226]:
# Save this as a csv file

import pandas as pd

# Save the DataFrame as a CSV file
extracted_data_yelp.to_csv('C:\\Users\\affuy\\Documents\\Data_Sets\\df_Yelp.csv', index=False)
